# Notebook to compare SWE DataSources 

# Step 0 Set up Notebook

In [1]:
import pandas as pd
import boto3
import s3fs
import xarray as xr
import matplotlib.pyplot as plt
import io
import requests
from io import StringIO
from snowML.datapipe import data_utils as du 

# Step 1 - Define Functions to Upload Data for One Huc

In [ ]:
def get_UA_data(huc): 
    f = "model
    b = "model-ready"
    df = s3_to_df(f, b)
    return df

In [ ]:
def get_Snowtel(huc): 
    

In [ ]:
def get_UCLA(huc): 
    